# Summary:

## Goal:
Given the data of music albums from [CSM](https://www.commonsensemedia.org/music-reviews), the purpose of this notebook is to find the corresponding album identifiers in Spotify.    

## Tools:
-  Spotify API and its python interface Spotipy.
-  Python regular expression to clean string and match

## Outcomes:
I found 1748 album ids out of 2206 albums.

# Import modules and credentials

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spotipy
import os
import sys
import re
import time 
%matplotlib inline

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from dotenv import load_dotenv
%load_ext dotenv
%dotenv

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


client_id = os.environ['SPOTIFY_CLIENT_ID']
client_secret = os.environ['SPOTIFY_CLIENT_SECRET']
spotify = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(client_id = client_id, client_secret=client_secret))

# Load  album age ratings

In [2]:
album_ratings = pd.read_csv("./data/Album_Ratings.csv")
print ("Number of albums: ", album_ratings.shape[0])
display(album_ratings.sample(3))

Number of albums:  2206


,Title,Age,Age Group,Artist,Year,Description
1977,Greatest Hits...So Far!!!,15,Teenager,Pink,2010,"Some mature themes -- but lots of positivity,..."
1178,Now That's What I Call Music! 17,12,Young Teen,Various Artists,2004,You've heard it all before.
1857,The Doors,14,Young Teen,The Doors,1967,"Morrison & Co.'s debut: dark, hypnotic all-ti..."


# Search albums using SpotifyAPI

In [ ]:
# Example: write search query

album_name = "Can a Jumbo Jet Sing the Alphabet?"
artist_name = "Hap Palmer"

results = spotify.search(q="album:" + album_name + " artist:" + artist_name, type='album,track')

print (results['albums']['items'][0]['name'])
print (results['albums']['items'][0]['id'])
print (results['albums']['items'][0]['artists'])


#  Define string-matchinig methods

In [3]:
from tabulate import tabulate


def isMatch(str1, str2, clean_str = None):
    
    """
    Decide whether str1 and str2 matched in a loose sense.
    """
    if (str1.lower() in str2.lower()) or (str2.lower() in str1.lower()):
        return True
    
    if clean_str:
        str1, str2 = clean_str(str1), clean_str(str2)
        return (str1 in str2) or (str2 in str1)
        


def manualMatch(title1, title2, artist1, artist2):
    """
    Let users decide whether (title1, artist1) and (title2, artist2) is a match.
    """

    

    display(pd.DataFrame([[title1, artist1],[title2, artist2]], index = ['query input','output']))
    
    match = input("Is it match? (y/n)")
    return (match.lower()=='y')

#  Search Albums: Found 953 albums

In [4]:
def search_album(album_name, artist_name, loose_query = False, manual_match = False, clean_str = None):
    
    if clean_str:
        album_name, artist_name = clean_str(album_name), clean_str(artist_name)
    
    if loose_query:
        results = spotify.search(q=album_name, type='album')  # slower
    else:
        results = spotify.search(q="album:" + album_name + " artist:" + artist_name, type='album') # faster

    for item in results['albums']['items']:
        album_name_spotify = item['name']
        album_id = item['id']
        artists = [artist['name'] for artist in item['artists']]
        
        if manual_match:
            for artist in artists:
                search_terms = [album_name, album_name_spotify, artist_name, artist]
                if manualMatch(*search_terms):
                    return [album_id, album_name_spotify, artists]
        
        elif isMatch(album_name, album_name_spotify):
            for artist in artists: 
                if isMatch(artist_name, artist, clean_str):
                    return [album_id, album_name_spotify, artists]
                 

In [5]:
results = []
t1 = time.time()
    
for (i, row) in album_ratings.iterrows():
    
    if i%300 ==0 :
        time_elapsed = (time.time()-t1)/60
        print (f"Processing {i}th row.  Time elapsed: {time_elapsed:.2f} minutes. Found {len(results)} albums.")
        
    album_name, artist_name = row.Title, row.Artist
   
    if not 'cd single' in album_name.lower():
        try:
            result = search_album(album_name, artist_name)

            if result: 
                results.append((album_name, artist_name, *result))
        except:
            print("(i, album_name, artist_name)=", (i,album_name, artist_name))
            print("Unexpected error:", sys.exc_info()[0])
            break
    
    
print (f"Found {len(results)} Albums.")

albums_id_1 = pd.DataFrame(results,columns=['Album_Name','Artist','Album_ID','Album_Name_Spotify','Artists_Spotify'])

Processing 0th row.  Time elapsed: 0.00 minutes. Found 0 albums.
Processing 300th row.  Time elapsed: 0.58 minutes. Found 157 albums.
Processing 600th row.  Time elapsed: 1.03 minutes. Found 252 albums.
Processing 900th row.  Time elapsed: 1.37 minutes. Found 358 albums.
Processing 1200th row.  Time elapsed: 1.67 minutes. Found 452 albums.
Processing 1500th row.  Time elapsed: 2.12 minutes. Found 607 albums.
Processing 1800th row.  Time elapsed: 2.53 minutes. Found 758 albums.
Processing 2100th row.  Time elapsed: 2.88 minutes. Found 875 albums.
Found 953 Albums.


In [6]:
#albums_id_1.to_csv("./data/albums_id_1.csv", index=False)

albums_id_1 = pd.read_csv("./data/albums_id_1.csv")
print ("Number of albums found: ", albums_id_1.shape[0])

# Albums name 
mask = albums_id_1['Album_Name']!=albums_id_1['Album_Name_Spotify']
display(albums_id_1[mask].sample(3))

Number of albums found:  953


,Album_Name,Artist,Album_ID,Album_Name_Spotify,Artists_Spotify
124,Parades and Panoramas,Dan Zanes,5ZagB9RVFTD7gXfpf1w7Yc,Parades And Panoramas: 25 Songs Collected By C...,['Dan Zanes']
252,I Don't Want to Go to School,The Naked Brothers Band,7944vq6z7A1SE3MoflJqvY,I Don't Want To Go To School,['The Naked Brothers Band']
731,Kevin Lyttle,Kevin Lyttle,0OWwtYwEZFMquSeHGNv3cw,Kevin Lyttle (US Domestic Release),['Kevin Lyttle']


# Search Albums with clean string, loose search: Found  125 albums

In [5]:
def cleanStr(s):
   
    """
    Clean input string by keeping only alphabet, numerics, and white space. Then lowercasing.
    """
    
    # https://stackoverflow.com/a/70310018
    
    pattern = "\(.*\)"
    pattern += "|Original Motion Picture Soundtrack|Original Soundtrack|Soundtrack"
    pattern += "|Soundtrack Album|Movie Soundtrack|Music from the Motion Picture|Original Motion Picture"
    s = re.sub(pattern, "", s, flags = re.IGNORECASE).lower().strip()
    
    #del_chars = ''.join(c for c in map(chr, range(256)) if not (c.isalnum()|c.isspace()))
    #del_map = str.maketrans('', '', del_chars)
    #scrunched = s.translate(del_map).lower().strip()
                        
    #return scrunched
    return s

In [136]:
results = []
t1 = time.time()
found = albums_id_1[['Album_Name','Artist']].values.tolist() 

for (i, row) in album_ratings.iterrows():
    
    if i%200 ==0 :
        time_elapsed = (time.time()-t1)/60
        print (f"Processing {i}th row.  Time elapsed: {time_elapsed:.2f} minutes. Found {len(results)} albums.")
        
    album_name, artist_name = row.Title, row.Artist
    
    if [album_name, artist_name] in found:  # ignore albums already found
        continue
    
    
    if not 'cd single' in album_name.lower():
        result = search_album(album_name, artist_name, loose_query = True, clean_str = cleanStr)
        if result: 
            results.append((album_name, artist_name, *result))
      
    
print (f"Found {len(results)} Albums.")

albums_id_2 = pd.DataFrame(results,columns=['Album_Name','Artist','Album_ID','Album_Name_Spotify','Artists_Spotify'])

Processing 0th row.  Time elapsed: 0.00 minutes. Found 0 albums.
Processing 200th row.  Time elapsed: 0.21 minutes. Found 6 albums.
Processing 400th row.  Time elapsed: 0.39 minutes. Found 18 albums.
Processing 600th row.  Time elapsed: 0.57 minutes. Found 52 albums.
Processing 800th row.  Time elapsed: 0.69 minutes. Found 72 albums.
Processing 1000th row.  Time elapsed: 0.78 minutes. Found 79 albums.
Processing 1200th row.  Time elapsed: 0.85 minutes. Found 85 albums.
Processing 1400th row.  Time elapsed: 0.93 minutes. Found 96 albums.
Processing 1600th row.  Time elapsed: 0.98 minutes. Found 106 albums.
Processing 1800th row.  Time elapsed: 1.06 minutes. Found 113 albums.
Processing 2000th row.  Time elapsed: 1.13 minutes. Found 120 albums.
Processing 2200th row.  Time elapsed: 1.19 minutes. Found 125 albums.
Found 125 Albums.


In [7]:
#albums_id_2.to_csv("./data/albums_id_2.csv", index=False)

albums_id_2 = pd.read_csv("./data/albums_id_2.csv")
print ("Number of albums found: ", albums_id_2.shape[0])

# Albums name 
mask = albums_id_2['Album_Name']!=albums_id_2['Album_Name_Spotify']
display(albums_id_2[mask].sample(3))

Number of albums found:  125


,Album_Name,Artist,Album_ID,Album_Name_Spotify,Artists_Spotify
28,A Cinderella Story Soundtrack,Various Artists,5fDujJMK88H9vZYRjVb2Bz,A Cinderella Story: Once Upon A Song (Original...,['Various Artists']
95,The Breakfast Club: Original Motion Picture So...,Various Artists,2yrEJ5xNvTdEEFuTtcku9M,The Breakfast Club,['Various Artists']
69,Scooby-Doo 2: Monsters Unleashed Soundtrack,Various Artists,5BobIMS3CojORsCYNeVky7,Scooby-Doo 2: Monsters Unleashed,['Various Artists']


# Search Singles:  Found 567 singles

In [8]:
def search_single(album_name, artist_name, loose_query = False, manual_match = False):
    
    track_name = album_name.lower().replace("(cd single)","").replace("\"","").strip()
    
    if loose_query:
        results =  spotify.search(q="track:" + track_name, type='track')
    else:
        results =  spotify.search(q="track:" + track_name + " artist:" + artist_name, type='track')
        
    for item in results['tracks']['items']:
        track_name_spotify = item['name']
        track_id = item['id']
        artists = [artist['name'] for artist in item['artists']]
                
        if manual_match:
            for artist in artists:
                search_terms = [album_name, track_name_spotify, artist_name, artist]         
                if manualMatch(*search_terms):
                    return [track_id, track_name_spotify, artists]
        
        elif isMatch(track_name, track_name_spotify):
            for artist in artists: 
                if isMatch(artist, artist_name):
                    return [track_id, track_name_spotify, artists]


In [140]:
results = []
t1 = time.time()

found = albums_id_1[['Album_Name','Artist']].values.tolist() + albums_id_2[['Album_Name','Artist']].values.tolist()

for (i, row) in album_ratings.iterrows():
    
    if i%400 ==0 :
        time_elapsed = (time.time()-t1)/60
        print (f"Processing {i}th row.  Time elapsed: {time_elapsed:.2f} minutes. Found {len(results)} singles.")
        
    album_name, artist_name = row.Title, row.Artist
    
    if ([album_name, artist_name] not in found) and ('cd single' in album_name.lower()):  
        result = search_single(album_name, artist_name)
        if result: 
            results.append((album_name, artist_name, *result))
        
        else:
            result = search_single(album_name, artist_name, loose_query = True)
            if result: 
                results.append((album_name, artist_name, *result))

    
print (f"Found {len(results)} Singles.")

singles_id_1 = pd.DataFrame(results,columns=['Album_Name','Artist','Track_ID','Track_Name_Spotify','Artists_Spotify'])

Processing 0th row.  Time elapsed: 0.00 minutes. Found 0 singles.
Processing 400th row.  Time elapsed: 0.02 minutes. Found 7 singles.
Processing 800th row.  Time elapsed: 0.35 minutes. Found 129 singles.
Processing 1200th row.  Time elapsed: 0.72 minutes. Found 295 singles.
Processing 1600th row.  Time elapsed: 1.00 minutes. Found 397 singles.
Processing 2000th row.  Time elapsed: 1.36 minutes. Found 526 singles.
Found 567 Singles.


In [141]:
singles_id_1.sample(3)

,Album_Name,Artist,Track_ID,Track_Name_Spotify,Artists_Spotify
209,"""I Found You"" (CD Single)",The Wanted,1CRY4X2b8X7X10EUdPUutw,I Found You,[The Wanted]
101,"""Burnin' Up"" (CD single)",Jonas Brothers,2VEsmoek0sol9MnJFyoG9e,Burnin' Up,[Jonas Brothers]
359,"""Troublemaker (feat. Flo Rida)"" (CD Single)",Olly Murs,6s8nHXTJVqFjXE4yVZPDHR,Troublemaker (feat. Flo Rida),"[Olly Murs, Flo Rida]"


In [10]:
#singles_id_1.to_csv("./data/singles_id_1.csv", index=False)

singles_id_1 = pd.read_csv("./data/singles_id_1.csv")
print ("Number of singles found: ", singles_id_1.shape[0])

display(singles_id_1.sample(3))

Number of singles found:  567


,Album_Name,Artist,Track_ID,Track_Name_Spotify,Artists_Spotify
122,"""Magic"" (CD Single)",Coldplay,23khhseCLQqVMCIT1WMAns,Magic,['Coldplay']
96,"""Burn"" (CD Single)",Ellie Goulding,0xMd5bcWTbyXS7wPrBtZA6,Burn,['Ellie Goulding']
442,"""Swagga Like Us"" (CD single)","Jay-Z, Kanye West, Lil Wayne, T.I.",6N6bpwA4iiq7HtKToeVH0c,Swagga Like Us,"['JAY-Z', 'T.I.', 'Kanye West', 'Lil Wayne']"


# Semi-Manual Search:  Display query result and decide match or not

In [11]:
results = []
t1 = time.time()

found = albums_id_1[['Album_Name','Artist']].values.tolist() + albums_id_2[['Album_Name','Artist']].values.tolist()
found += singles_id_1[['Album_Name', 'Artist']].values.tolist()

print (f"Found {len(found)} albums; {len(album_ratings)-len(found)} albums not found yet.")

Found 1645 albums; 561 albums not found yet.


In [ ]:
for (i, row) in album_ratings.iterrows():
    
    if i%400 ==0 :
        time_elapsed = (time.time()-t1)/60
        print (f"Processing {i}th row.  Time elapsed: {time_elapsed:.2f} minutes. Found {len(results)} singles.")
        
    album_name, artist_name = row.Title, row.Artist
    
    if ([album_name, artist_name] not in found) and ('cd single' not in album_name.lower()):  
        result = search_album(album_name, artist_name, loose_query = True, manual_match = True)
        if result: 
            results.append((album_name, artist_name, *result))
        
    
print (f"Semi-manually Found {len(results)} albums.")

albums_id_3 = pd.DataFrame(results,columns=['Album_Name','Artist','Track_ID','Track_Name_Spotify','Artists_Spotify'])

Processing 0th row.  Time elapsed: 16.71 minutes. Found 0 singles.


,0,1
0,A Child's Celebration of Song 2,Various Artists
1,Celebration of Half Century Collection of Chil...,Various Artists


Is it match? (y/n)n


,0,1
0,World Party!,Various Artists
1,World Party,Goodie Mob


Is it match? (y/n)y


...

,0,1
0,Los Lonely Boys,Los Lonelyboys
1,Los Lonely Boys,Los Lonely Boys


Is it match? (y/n)y
Processing 800th row.  Time elapsed: 39.49 minutes. Found 94 singles.


,0,1
0,The Beatles: 1,The Beatles
1,The Beatles 1967 - 1970 (Remastered),The Beatles


Is it match? (y/n)n


,0,1
0,The Beatles: 1,The Beatles
1,1 (Remastered),The Beatles


In [167]:
albums_id_3 = pd.DataFrame(results,columns=['Album_Name','Artist','Track_ID','Track_Name_Spotify','Artists_Spotify'])

In [12]:
#albums_id_3.to_csv("./data/albums_id_3.csv", index=False)

albums_id_3 = pd.read_csv("./data/albums_id_3.csv")

albums_id_3.sample(3)

,Album_Name,Artist,Track_ID,Track_Name_Spotify,Artists_Spotify
34,Welcome to the Village,"Aaron Nigel Smith, One World Chorus",0GJLKSh0xldiGuFRFE6VG3,Welcome to the Village!,['Aaron Nigel Smith & One World Chorus']
6,Swing Around the World,Various Artists,1X70XfhXt2okvamjvuZXVK,Swing Around the World,['Fleur Seule']
0,World Party!,Various Artists,6yRkvrlMyH6xE6mTzLXomz,World Party,['Goodie Mob']


In [13]:
results = []
t1 = time.time()

found = albums_id_1[['Album_Name','Artist']].values.tolist() + albums_id_2[['Album_Name','Artist']].values.tolist()
found += singles_id_1[['Album_Name', 'Artist']].values.tolist()
found += albums_id_3[['Album_Name','Artist']].values.tolist()

print (f"Found {len(found)} albums; {len(album_ratings)-len(found)} albums not found yet.")

Found 1745 albums; 461 albums not found yet.


In [ ]:
for (i, row) in album_ratings[200:].iterrows():
    
    if i%400 ==0 :
        time_elapsed = (time.time()-t1)/60
        print (f"Processing {i}th row.  Time elapsed: {time_elapsed:.2f} minutes. Found {len(results)} singles.")
        
    album_name, artist_name = row.Title, row.Artist
    
    if ([album_name, artist_name] not in found) and ('cd single' not in album_name.lower()):  
        result = search_album(album_name, artist_name, loose_query = True, manual_match = True)
        if result: 
            results.append((album_name, artist_name, *result))
        
    
print (f"Semi-manually Found {len(results)} albums.")

albums_id_4 = pd.DataFrame(results,columns=['Album_Name','Artist','Track_ID','Track_Name_Spotify','Artists_Spotify'])

,0,1
query input,Santa Songs,Hullabaloo
output,Santa Shark & More Kids Christmas Songs,Super Simple Songs


Is it match? (y/n)n


,0,1
query input,Santa Songs,Hullabaloo
output,Santa Songs,Dan Crow


Is it match? (y/n)n


,0,1
query input,Santa Songs,Hullabaloo
output,Red Dress Special - Pow-Wow Songs Recorded Liv...,Young Spirit


...

,0,1
query input,Social Studies,Rhythm Rhyme Results
output,Social Studies (Deluxe Edition),Body Language


Is it match? (y/n)n


,0,1
query input,Social Studies,Rhythm Rhyme Results
output,Social Studies,Loudon Wainwright III


Is it match? (y/n)n


,0,1
query input,Social Studies,Rhythm Rhyme Results
output,Social Studies: an Introduction to Charles Atlas,Charles Atlas


In [15]:
albums_id_4 = pd.DataFrame(results,columns=['Album_Name','Artist','Track_ID','Track_Name_Spotify','Artists_Spotify'])

In [16]:
albums_id_4.to_csv("./data/albums_id_4.csv", index=False)

albums_id_4 = pd.read_csv("./data/albums_id_4.csv")

albums_id_4.sample(3)

,Album_Name,Artist,Track_ID,Track_Name_Spotify,Artists_Spotify
2,Dev2.0,Dev2.0,5yOLkUelCWvz7pNb3Uyvu6,Devo 2.0,['Devo 2.0']
1,Making Silly Faces,Nick Deysher,2MPgd67X25SmFa01Gj6XRZ,Making Silly Faces,['In The Nick of Time']
0,The Fresh Beat Band: Music from the Hit TV Sho...,The Fresh Beat Band,6jXrznEcmVQ741Kq7zDbQg,The Fresh Beat Band Vol 2.0: More Music From T...,['The Fresh Beat Band']


# Summary: number of albums found

In [3]:
import pandas as pd
albums_id_1 = pd.read_csv("../data/albums_id_1.csv")
albums_id_2 = pd.read_csv("../data/albums_id_2.csv")

albums_id_3 = pd.read_csv("../data/albums_id_3.csv")
albums_id_4 = pd.read_csv("../data/albums_id_4.csv")
singles_id_1 = pd.read_csv("../data/singles_id_1.csv")

In [10]:
albums_df = pd.concat([albums_id_1, albums_id_2, albums_id_3, albums_id_4])
albums_df = albums_df.drop(columns=['Track_ID', 'Track_Name_Spotify'])
print (albums_df.shape[0], "albums")
print (singles_id_1.shape[0], "singles")
print ("total: albums + singles = ", albums_df.shape[0] + singles_id_1.shape[0])

display(albums_df.sample(3))
display(singles_id_1.sample(3))

1181 albums
567 singles
total: albums + singles =  1748


,Album_Name,Artist,Album_ID,Album_Name_Spotify,Artists_Spotify
4,cELLAbration! A Tribute to Ella Jenkins,Various Artists,NaN,NaN,['Various Artists']
920,Curtain Call,Eminem,5qENHeCSlwWpEzb25peRmQ,Curtain Call: The Hits (Deluxe Edition),['Eminem']
43,Sharpay's Fabulous Adventure Soundtrack,"Ashley Tisdale, Various Artists",7vDbqYD54hHrZ1yjmPljS4,Sharpay's Fabulous Adventure,['Various Artists']


,Album_Name,Artist,Track_ID,Track_Name_Spotify,Artists_Spotify
13,"""The Time of My Life"" (CD single)",David Cook,6avyV3xlz2UFeiAUZ0SqyT,The Time of My Life,['David Cook']
328,"""I Am the Champion"" (CD single)",B.o.B.,57gArW5xYHNXaw2am8s5eG,I Am the Champion,['B.o.B']
224,"""Cups"" (CD Single)",Anna Kendrick,6mH3qVIeOsnQIAho5eWwhH,Cups (Pitch Perfect’s “When I’m Gone”) - Pop V...,['Anna Kendrick']


In [9]:
albums_df.to_csv("../data/album_ids.csv", index = False)
singles_id_1.to_csv("../data/single_ids.csv", index = False)